In [1]:
import requests
import pandas as pd
import time

# Establish connection to Yahoo Finance
yahoo_finance_url = 'https://au.finance.yahoo.com/quote/%s/history?period1=%s&period2=%s&interval=1d&filter=history&frequency=1d' 
# July 1, 2018 starting_date = '1530403200'
# June 30, 2019 ending_day = '1561939199'
timestamp = ['1530403200', '1536710400', '1543017600', '1549324800', '1555632000', '1561939199']
# timestamp = [20183.7.31, 2018.9.12, 2018.11.24, 2019.2.5, 2019.4.19, 2019.6.30]


def fetch_Yahoo_Finance(security, timestamp):
    All_DataFrame = None
    for starting in range(0, len(timestamp)-1):
        each_DataFrame = fetch_Yahoo_Finance_part(security, timestamp[starting], timestamp[starting+1])
        All_DataFrame = pd.concat([All_DataFrame, each_DataFrame], axis = 0)
    #All_DataFrame.set_index(All_DataFrame['Date'], inplace = True)
    #All_DataFrame.sort_index(ascending = True, inplace = True)
    temp = All_DataFrame.sort_values(by = 'Date', ascending = True)
    tt = temp.reset_index(drop = 'True')
    return tt

# Return each security's Ri
def fetch_Yahoo_Finance_part(security, starting, ending):
    yahoo_hist_price_page = requests.get(yahoo_finance_url % (security, starting, ending))    
    # Read historical summary statistics table from Yahoo Finance and clean the DataFrame
    
    # To get every rows but the last row
    yahoo_hist_price_DataFrame = pd.read_html(yahoo_hist_price_page.text)[0].iloc[:-1]
    # Drop these dividend records from the DataFrame
    yahoo_hist_price_DataFrame = yahoo_hist_price_DataFrame[~yahoo_hist_price_DataFrame['Adj. close**'].str.contains('Dividend')] 
    # We only need its Adj. close**
    for column in yahoo_hist_price_DataFrame.columns:
        if column != 'Adj. close**':
            yahoo_hist_price_DataFrame.drop([column], axis = 1)
    # Convert all relevant strings to floats/numbers. Reset to 'NaN' for those shouldn't be converted.
    for column in yahoo_hist_price_DataFrame.columns:
        if column != 'Date':
            yahoo_hist_price_DataFrame[column] = pd.to_numeric(yahoo_hist_price_DataFrame[column], errors='coerce')
    
    # Add security and daily stock return to the DataFrame
    yahoo_hist_price_DataFrame['%s Ri'% security] = yahoo_hist_price_DataFrame['Adj. close**'].pct_change(periods=1)
    
    # Convert String to Date
    yahoo_hist_price_DataFrame['Date'] = pd.to_datetime(yahoo_hist_price_DataFrame['Date'])
    temp = yahoo_hist_price_DataFrame.loc[:,['Date', '%s Ri'% security]]
    return temp




In [2]:
start = time.time()
security_test = fetch_Yahoo_Finance('^GSPC', timestamp)
end = time.time()
ti = end - start
print(ti, security_test)

17.042591094970703           Date  ^GSPC Ri
0   2018-07-02  0.004972
1   2018-07-03 -0.008547
2   2018-07-05 -0.008410
3   2018-07-06 -0.008746
4   2018-07-09 -0.003461
..         ...       ...
245 2019-06-24  0.009587
246 2019-06-25  0.001236
247 2019-06-26 -0.003809
248 2019-06-27 -0.005724
249 2019-06-28       NaN

[250 rows x 2 columns]


In [3]:
# Use for Rm
def SP500():
    time1 = time.time()
    sp500 = fetch_Yahoo_Finance('^GSPC', timestamp)
    time2 = time.time()
    sp500.rename(columns = {'^GSPC Ri':'^GSPC Rm'}, inplace = True)
    print("Fetch Rm:SP500 sucessful. Spend time %s" % (time2 - time1))
    print()
    return sp500

Rm = SP500()
Rm

Fetch Rm:SP500 sucessful. Spend time 18.894108772277832



,Date,^GSPC Rm
0,2018-07-02,0.004972
1,2018-07-03,-0.008547
2,2018-07-05,-0.008410
3,2018-07-06,-0.008746
4,2018-07-09,-0.003461
...,...,...
245,2019-06-24,0.009587
246,2019-06-25,0.001236
247,2019-06-26,-0.003809
248,2019-06-27,-0.005724


In [4]:
# Use for Rf
def US_Treasure():
    time1 = time.time()
    US_treasury_URL = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=%s'
    year1 = '2018'
    year2 = '2019'
    US_treasury_hist_price_page1 = requests.get(US_treasury_URL % year1)
    US_treasury_hist_price_page2 = requests.get(US_treasury_URL % year2)

    #Read Table from the US Treasury webpage
    US_treasury_hist_price_DataFrame1 = pd.read_html(US_treasury_hist_price_page1.text)[1]
    US_treasury_hist_price_DataFrame2 = pd.read_html(US_treasury_hist_price_page2.text)[1]

    #Convert string to date
    US_treasury_hist_price_DataFrame1['Date'] = pd.to_datetime(US_treasury_hist_price_DataFrame1['Date'])
    US_treasury_hist_price_DataFrame2['Date'] = pd.to_datetime(US_treasury_hist_price_DataFrame2['Date'])
    
    temp1 = US_treasury_hist_price_DataFrame1.loc[:,['Date', '1 mo']]
    temp2 = US_treasury_hist_price_DataFrame2.loc[:,['Date', '1 mo']]
    
    #Combine US treasure of 2018 and 2019
    Combined_US_Treasure = None
    Combined_US_Treasure = pd.concat([temp1, temp2], axis = 0)
    
    time2 = time.time()
    print("Fetch Rf:US Treasure sucessful. Spend time %s" % (time2 - time1))
    print()
    return Combined_US_Treasure


Rf = US_Treasure()
Rf

Fetch Rf:US Treasure sucessful. Spend time 4.8089563846588135



,Date,1 mo
0,2018-01-02,1.29
1,2018-01-03,1.29
2,2018-01-04,1.28
3,2018-01-05,1.27
4,2018-01-08,1.30
...,...,...
245,2019-12-24,1.55
246,2019-12-26,1.59
247,2019-12-27,1.56
248,2019-12-30,1.51


In [5]:
def X1(Rm, Rf):
    time1 = time.time()
    X1_DataFrame = pd.merge(Rm, Rf, on = 'Date')
    X1_DataFrame['Rm_Rf'] = X1_DataFrame['^GSPC Rm'] - X1_DataFrame['1 mo']/100/365
    temp = X1_DataFrame.loc[:,['Date', 'Rm_Rf']] 
    time2 = time.time()
    print("Obtain X1 DataFrame Sucessful. Spend time %s." % (time2 - time1))
    print()
    return temp

xx1 = X1(Rm, Rf)
xx1

Obtain X1 DataFrame Sucessful. Spend time 0.013962268829345703.



,Date,Rm_Rf
0,2018-07-02,0.004920
1,2018-07-03,-0.008599
2,2018-07-05,-0.008461
3,2018-07-06,-0.008797
4,2018-07-09,-0.003512
...,...,...
243,2019-06-24,0.009529
244,2019-06-25,0.001178
245,2019-06-26,-0.003866
246,2019-06-27,-0.005784


In [6]:
def SP500_cap_range_list():
    time1 = time.time()
    #web has already sorted by cap
    stocks_pool_url = 'https://www.slickcharts.com/sp500'                 
    stocks_pool_page = requests.get(stocks_pool_url)
    stocks_pool_DataFrame = pd.read_html(stocks_pool_page.text)[0]
    
    #construct the target list and list_for_test
    stocks_pool_DataFrame_test = stocks_pool_DataFrame[31:61]['Symbol']
    stocks_pool_DataFrame = stocks_pool_DataFrame[0:31]['Symbol']
    security_list_test = stocks_pool_DataFrame_test.tolist()
    security_list = stocks_pool_DataFrame.tolist()
    security_list.remove('BRK.B')
    time2 = time.time()
    print("Obtained Y and X2 security list sucessful.Spend time %s." %(time2 - time1))
    print()
    # security_list_test是回归方程等式右边， security_list是等式左边
    return security_list_test, security_list

def fetch_vol(security_list):
    print("Obtaing X3 security list...")
    time_1 = time.time()
    #initialization
    security_vol_dict = {}
    security_list_vol = []
    #use avg volume
    yahoo_vol_url = 'https://au.finance.yahoo.com/quote/%s?p=%s'               
    
    for security in security_list:
        time1 = time.time()
        #fetch the page
        yahoo_vol_page = requests.get(yahoo_vol_url % (security,security))
        yahoo_vol_data_frame = pd.read_html(yahoo_vol_page.text)[0]
        #fetch the Num and add them into a dict
        vol = yahoo_vol_data_frame.iloc[7,1]
        vol_1 = pd.to_numeric(vol)
        security_vol_dict[security] = vol_1
        time2 = time.time()
        print("get %s, spend time %s" % (security, (time2 - time1)))
    #sort the security_volume dict by the volume
    vol_rank = sorted(security_vol_dict.items(),key=lambda x:x[1])
    #fetch the name of security and construct the target list
    for key,value in vol_rank:
        security_list_vol.append(key)
        #print("%s appended" % key)
    time_2 = time.time()
    print("Obtain X3 security list sucessful. Spend time %s" % (time_2 - time_1))
    print()
    return  security_list_vol

In [7]:
Security_List = SP500_cap_range_list()
Y_security = Security_List[0]
X_security = Security_List[1]
X_security_ = fetch_vol(X_security)

Obtained Y and X2 security list sucessful.Spend time 2.980813980102539.

Obtaing X3 security list...
get MSFT, spend time 2.5777127742767334
get AAPL, spend time 3.4386937618255615
get AMZN, spend time 2.9596939086914062
get FB, spend time 4.088997840881348
get GOOGL, spend time 3.217681884765625
get GOOG, spend time 3.5555620193481445
get JNJ, spend time 2.5926012992858887
get V, spend time 2.6383326053619385
get JPM, spend time 4.310945749282837
get UNH, spend time 2.7953603267669678
get PG, spend time 4.0470335483551025
get INTC, spend time 4.270660877227783
get HD, spend time 2.906954526901245
get MA, spend time 6.074565887451172
get T, spend time 4.309194564819336
get VZ, spend time 3.103559970855713
get DIS, spend time 5.65755558013916
get BAC, spend time 4.105039358139038
get NVDA, spend time 3.805481195449829
get PFE, spend time 2.2987048625946045
get MRK, spend time 3.8553543090820312
get XOM, spend time 3.306596040725708
get CSCO, spend time 3.22674822807312
get NFLX, spend t

In [9]:
X_security, X_security_, Y_security

(['MSFT',
  'AAPL',
  'AMZN',
  'FB',
  'GOOGL',
  'GOOG',
  'JNJ',
  'V',
  'JPM',
  'UNH',
  'PG',
  'INTC',
  'HD',
  'MA',
  'T',
  'VZ',
  'DIS',
  'BAC',
  'NVDA',
  'PFE',
  'MRK',
  'XOM',
  'CSCO',
  'NFLX',
  'PEP',
  'ADBE',
  'CMCSA',
  'KO',
  'CVX',
  'WMT'],
 ['GOOG',
  'GOOGL',
  'ADBE',
  'AMZN',
  'UNH',
  'MA',
  'PEP',
  'HD',
  'NFLX',
  'WMT',
  'JNJ',
  'PG',
  'MRK',
  'V',
  'CVX',
  'NVDA',
  'VZ',
  'KO',
  'DIS',
  'FB',
  'JPM',
  'CMCSA',
  'PFE',
  'CSCO',
  'INTC',
  'XOM',
  'T',
  'AAPL',
  'MSFT',
  'BAC'],
 ['PYPL',
  'ABT',
  'ABBV',
  'CRM',
  'BMY',
  'MCD',
  'TMO',
  'COST',
  'AMGN',
  'MDT',
  'ACN',
  'NKE',
  'LLY',
  'UNP',
  'NEE',
  'ORCL',
  'PM',
  'C',
  'AVGO',
  'AMT',
  'IBM',
  'TXN',
  'WFC',
  'HON',
  'LIN',
  'DHR',
  'LMT',
  'LOW',
  'RTX',
  'GILD'])

In [10]:
# Combine all Ri and return Ri's average
def combine_Ri_DataFrame(security_list, timestamp):
    # Use the first security to initialize the first DataFrame
    print("Fetching Security %s %s......" % (1, security_list[0]))
    time_1 = time.time()
    Combined_Ri_DataFrame = fetch_Yahoo_Finance(security_list[0], timestamp)
    time_2 = time.time()
    print("Fetching %s sucessful. Spend time %s" % (security_list[0], (time_2 - time_1)))
    print()
    for i in range(1, len(security_list)):
        # Check whether the DataFrame has data and combine them
        time1 = time.time()
        print("Fetching Security %s %s......" % (i + 1, security_list[i]))
        security_Ri_DataFrame = fetch_Yahoo_Finance(security_list[i], timestamp)
        time2 = time.time()
        print("Fetch  %s sucessful, time spend %s" % (security_list[i], time2 - time1))
        Combined_Ri_DataFrame = pd.merge(Combined_Ri_DataFrame, security_Ri_DataFrame, on = 'Date')
        print("Combined  %s sucessful" % security_list[i])
        time3 = time.time()
        print("Obtaining current Ri Average....")
        Combined_Ri_DataFrame['Ri Average'] = Combined_Ri_DataFrame.mean(axis = 1)
        for column in Combined_Ri_DataFrame.columns:
            if column != 'Ri Average':
                Combined_Ri_DataFrame.drop([column], axis = 1)
        time4 = time.time()
        print("Obtaining current Ri Average sucessful. Spend time %s" %(time4 - time3))
        print()
        
        
    # Create column to store the average Ri for all the securities
    #print("Calculate Ri.....")
    #Combined_Ri_DataFrame['Ri Average'] = Combined_Ri_DataFrame.mean(axis = 1)
    #print("Calculate Ri sucessful")
    temp = Combined_Ri_DataFrame.loc[:,['Date', 'Ri Average']]
    return temp


In [11]:
def X2(security_list, timestamp):
    time1 = time.time()
    list1 = security_list[0:int(len(security_list)/2)]
    list2 = security_list[int(len(security_list)/2):int(len(security_list))]
    R1 = combine_Ri_DataFrame(list1, timestamp)
    R1.rename(columns = {'Ri Average':'R1 Ri Average'}, inplace = True)
    R2 = combine_Ri_DataFrame(list2, timestamp)
    R2.rename(columns = {'Ri Average':'R2 Ri Average'}, inplace = True)
    R = pd.merge(R1, R2, on = 'Date')
    R['Weight'] = R['R1 Ri Average'] - R['R2 Ri Average']
    temp = R.loc[:,['Date', 'Weight']]
    time2 = time.time()
    print("X2 DataFrame create sucessful. Spend time %s" % (time2 - time1))
    print()
    return temp

tx21 = time.time()
xx2 = X2(X_security, timestamp)
tx22 = time.time()

print(tx22 - tx21)

Fetching Security 1 MSFT......
Fetching MSFT sucessful. Spend time 14.63321590423584

Fetching Security 2 AAPL......
Fetch  AAPL sucessful, time spend 15.122321367263794
Combined  AAPL sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.010113954544067383

Fetching Security 3 AMZN......
Fetch  AMZN sucessful, time spend 12.279988765716553
Combined  AMZN sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.0

Fetching Security 4 FB......
Fetch  FB sucessful, time spend 11.999879360198975
Combined  FB sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.005049943923950195

Fetching Security 5 GOOGL......
Fetch  GOOGL sucessful, time spend 11.460168600082397
Combined  GOOGL sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.009740114212036133

Fetching Security 6 GOOG......
Fetch  GOOG sucessful, time spend 14.033909

In [13]:
xx2

,Date,Weight
0,2018-07-02,0.005056
1,2018-07-03,-0.004098
2,2018-07-05,-0.001184
3,2018-07-06,-0.002083
4,2018-07-09,0.002837
...,...,...
245,2019-06-24,0.003561
246,2019-06-25,-0.001961
247,2019-06-26,0.001718
248,2019-06-27,-0.002517


In [14]:
def X3(security_list,timestamp):
    time1 = time.time()
    list1 = security_list[0:int(len(security_list)/2)]
    list2 = security_list[int(len(security_list)/2):int(len(security_list))]
    R1 = combine_Ri_DataFrame(list1, timestamp)
    R1.rename(columns = {'Ri Average':'R1 Ri Average'}, inplace = True)
    R2 = combine_Ri_DataFrame(list2, timestamp)
    R2.rename(columns = {'Ri Average':'R2 Ri Average'}, inplace = True)
    R = pd.merge(R1, R2, on = 'Date')
    R['Volume'] = R['R1 Ri Average'] - R['R2 Ri Average']
    temp = R.loc[:,['Date', 'Volume']]
    time2 = time.time()
    print("X3 DataFrame create sucessful. Spend time %s" % (time2 - time1))
    print()
    return temp

tx31 = time.time()
xx3 = X3(X_security_, timestamp)
tx32 = time.time()
print(tx32 - tx31)

Fetching Security 1 GOOG......
Fetching GOOG sucessful. Spend time 11.782730340957642

Fetching Security 2 GOOGL......
Fetch  GOOGL sucessful, time spend 13.621653079986572
Combined  GOOGL sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.003989458084106445

Fetching Security 3 ADBE......
Fetch  ADBE sucessful, time spend 13.124340295791626
Combined  ADBE sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.003985166549682617

Fetching Security 4 AMZN......
Fetch  AMZN sucessful, time spend 14.534607648849487
Combined  AMZN sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.004017829895019531

Fetching Security 5 UNH......
Fetch  UNH sucessful, time spend 14.239357471466064
Combined  UNH sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.003989458084106445

Fetching Security 6 MA......
Fetch  MA sucessful, tim

In [15]:
xx3

,Date,Volume
0,2018-07-02,-0.001114
1,2018-07-03,-0.000658
2,2018-07-05,0.001333
3,2018-07-06,0.000939
4,2018-07-09,-0.004872
...,...,...
245,2019-06-24,0.003283
246,2019-06-25,0.008564
247,2019-06-26,-0.000490
248,2019-06-27,0.004153


In [16]:
def Y(security_list, timestamp, Rf):
    time1 = time.time()
    Ri = combine_Ri_DataFrame(security_list, timestamp)
    Y_DataFrame = pd.merge(Ri, Rf, on = 'Date')
    Y_DataFrame['Ri_Rf'] = Y_DataFrame['Ri Average'] - Y_DataFrame['1 mo']/100/365
    temp = Y_DataFrame.loc[:,['Date', 'Ri_Rf']]
    time2 = time.time()
    print("Y DataFrame create sucessful. Spend time %s" % (time2 - time1))
    print()
    return temp

yt1 = time.time()
yyy = Y(Y_security, timestamp, Rf)
yt2 = time.time()
print(yt2 - yt1)

Fetching Security 1 PYPL......
Fetching PYPL sucessful. Spend time 10.698163270950317

Fetching Security 2 ABT......
Fetch  ABT sucessful, time spend 11.648396730422974
Combined  ABT sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.003989219665527344

Fetching Security 3 ABBV......
Fetch  ABBV sucessful, time spend 10.800440549850464
Combined  ABBV sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.002991914749145508

Fetching Security 4 CRM......
Fetch  CRM sucessful, time spend 11.505350351333618
Combined  CRM sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.004018545150756836

Fetching Security 5 BMY......
Fetch  BMY sucessful, time spend 11.145170211791992
Combined  BMY sucessful
Obtaining current Ri Average....
Obtaining current Ri Average sucessful. Spend time 0.004986763000488281

Fetching Security 6 MCD......
Fetch  MCD sucessful, time spend

In [17]:
yyy

,Date,Ri_Rf
0,2018-07-02,0.003808
1,2018-07-03,-0.009391
2,2018-07-05,-0.009050
3,2018-07-06,-0.006323
4,2018-07-09,-0.005300
...,...,...
245,2019-06-24,0.012490
246,2019-06-25,0.004865
247,2019-06-26,-0.005895
248,2019-06-27,-0.006648


In [18]:
start = time.time()
Y_X = pd.merge(yyy, xx1, on = 'Date')
Y_X = pd.merge(Y_X, xx2, on = 'Date')
Y_X = pd.merge(Y_X, xx3, on = 'Date')
end = time.time()

In [19]:
Y_X

,Date,Ri_Rf,Rm_Rf,Weight,Volume
0,2018-07-02,0.003808,0.004920,0.005056,-0.001114
1,2018-07-03,-0.009391,-0.008599,-0.004098,-0.000658
2,2018-07-05,-0.009050,-0.008461,-0.001184,0.001333
3,2018-07-06,-0.006323,-0.008797,-0.002083,0.000939
4,2018-07-09,-0.005300,-0.003512,0.002837,-0.004872
...,...,...,...,...,...
245,2019-06-24,0.012490,0.009529,0.003561,0.003283
246,2019-06-25,0.004865,0.001178,-0.001961,0.008564
247,2019-06-26,-0.005895,-0.003866,0.001718,-0.000490
248,2019-06-27,-0.006648,-0.005784,-0.002517,0.004153


In [20]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [21]:
R_model = ols("""Ri_Rf ~ Rm_Rf""", data = Y_X).fit()
print(R_model.summary())
print(R_model.params)

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     3472.
Date:                Fri, 29 May 2020   Prob (F-statistic):          6.70e-146
Time:                        00:58:32   Log-Likelihood:                 1128.3
No. Observations:                 245   AIC:                            -2253.
Df Residuals:                     243   BIC:                            -2246.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -1.782      0.0

In [22]:
R_model = ols("""Ri_Rf ~ Rm_Rf + Weight""", data = Y_X).fit()
print(R_model.summary())
print(R_model.params)

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     1928.
Date:                Fri, 29 May 2020   Prob (F-statistic):          2.15e-149
Time:                        00:58:34   Log-Likelihood:                 1140.8
No. Observations:                 245   AIC:                            -2276.
Df Residuals:                     242   BIC:                            -2265.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003      0.000     -1.692      0.0

In [23]:
R_model = ols("""Ri_Rf ~ Rm_Rf + Weight + Volume""", data = Y_X).fit()
print(R_model.summary())
print(R_model.params)

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     1303.
Date:                Fri, 29 May 2020   Prob (F-statistic):          1.34e-148
Time:                        00:58:34   Log-Likelihood:                 1143.0
No. Observations:                 245   AIC:                            -2278.
Df Residuals:                     241   BIC:                            -2264.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0002      0.000     -1.659      0.0